In [284]:
import datetime
from collections import defaultdict

#from block_index import BlockIndex # Kyles original 
from block_index_remote import BlockIndex ##BB
from block_classifier import BlockClassifier

from tqdm import tqdm
import pandas as pd
import numpy as np

from config import Day1Config as d1

In [285]:
index = BlockIndex(read_only=True, table='day1')
classifier = BlockClassifier()

block_labels = defaultdict(lambda: defaultdict(int)) #Kyle's way to build the dictionary
blocks_with_locations = []
blocks_for_dataFrame = {
    'block_number': [],
    'extra_data': [],
    'location': []
}

blocks_to_parse = index.list_blocks_range(d1.start_block, d1.end_block, "day1")
total = d1.end_block - d1.start_block +2 ## includes both extremes
print(f" total blocks_to_parse: {total}")

csv = "block_number, location, extra_data\n"

 total blocks_to_parse: 5883


## Parse Blocks - convert extra_data to locations or pools

In [286]:
for block in tqdm(blocks_to_parse, total=total):
    #date = block.get_datetime().date()
    block_number = block.get_block_number()

    # first, try to label the block based on the extra data
    label = classifier.classify_extra_data(block.extra_data)
    if label is not None:
        label = 'extraData:' + label
    else:
        # if that doesn't work, label it based on the pool
        label = classifier.classify_miner(block.miner)
        if label is not None:
            label = 'pool:' + label
    # if neither work, call it 'unknown'
    if label is None:
        label = 'unknown'
    
    # kyle record it based on date or block_number
    #block_labels[date][label] += 1

    # this shows for each block
    #block_labels[block_number][label] += 1  #works
    
    ''' BB 
    blocks_with_locations.append( {
        'block_number': block_number,
        'extra_data': block.extra_data_decoded(),
        'location': label,
    })
    ##'''
    
    bdf = blocks_for_dataFrame
    bdf['block_number'].append(block_number)
    bdf['extra_data'].append(block.extra_data_decoded())
    bdf['location'].append(label)
    
    #csv += f"{block_number},{label},{block.extra_data_decoded()}\n"

100%|█████████▉| 5882/5883 [00:00<00:00, 59378.83it/s]


In [287]:
'''  Kyle   
df = pd.DataFrame(block_labels)#.T.sort_index()
print(df)
print("\ndf.T") # Transposes: turns row labels into columns
print(df.T)
print("\ndf.T.sort_index()")
print(df.T.sort_index())
#df.index.name = 'block_number'
#df.to_csv('output/block-labels.csv', float_format='%.0f')
'''
# create the DataFrame 
columns = {
    'extraData': blocks_for_dataFrame['extra_data'],
    'location': blocks_for_dataFrame['location'],
    'location_assigned': np.nan                      # prepare the empty colum
}
df = pd.DataFrame(columns, index=blocks_for_dataFrame['block_number'])
df.head()

,extraData,location,location_assigned
15531512,EthereumPPLNS/2miners_ASIA4,extraData:asia,NaN
15531513,lhpool.com-h15,unknown,NaN
15531514,七彩神仙鱼F !,pool:F2Pool,NaN
15531515,七彩神仙鱼F#!,pool:F2Pool,NaN
15531516,Flexpool/S1/HK - Boston,pool:Flexpool.io,NaN


In [288]:
# Other info
#df.set_index("block_number")
#df.index.name("block_number")
#print(df)
#print(f"\nindex: {df.index}")
#print(f"\nshape: {df.shape}")
#print(f"\ncolumns: {df.columns}")
#print(f"\ncount(): {df.count()}")
#print(f"\ninfo: {df.info()}")
#print(df.loc[[15531512]])  # to access a row by its index
#df['location'][15531512]   # to access a specific location of a specific block

## Setup locations dataframes for all labels (pools + extraData)
will generate a label_to_region_mix dataframe to use

In [289]:
# Kyles second step from estimate_emissions_factors.py
from emissions_utils import dot, normalize_row_to_one, read_csv_date

# load mapping from mining pool to distribution across regions, dropping unknown pools
miner_to_region_mix = pd.read_csv('input/miner-to-region-mix.csv').set_index('miner')
miner_to_region_mix = miner_to_region_mix.drop(columns=['url'])#, 'unknown'])
miner_to_region_mix = miner_to_region_mix.dropna(how='all')               # drops unknown pools
miner_to_region_mix = normalize_row_to_one(miner_to_region_mix.fillna(0)) #normalize = turn 95.0 into 0.95 and NaN into 0.0
miner_to_region_mix.head()

,europe,us,asia,china,russia,unknown
miner,,,,,,
2Miners,0.95,0.04,0.01,0.0,0.0,0.0
Bw Pool,0.32,0.10,0.58,0.0,0.0,0.0
Coinotron,0.92,0.08,0.00,0.0,0.0,0.0
DwarfPool,0.84,0.12,0.04,0.0,0.0,0.0
Ethermine,0.72,0.15,0.13,0.0,0.0,0.0


In [290]:
## TODO simplify and merge line 6 with the previous step and cell

# construct a mapping from block labels of miners to region mix
miner_label_to_region_mix = miner_to_region_mix.copy()
#rename the index to match the location including the "pool" part
miner_label_to_region_mix.index = ['pool:' + e for e in miner_label_to_region_mix.index] 
miner_label_to_region_mix#.head()

,europe,us,asia,china,russia,unknown
pool:2Miners,0.950000,0.040000,0.010000,0.0,0.0,0.0
pool:Bw Pool,0.320000,0.100000,0.580000,0.0,0.0,0.0
pool:Coinotron,0.920000,0.080000,0.000000,0.0,0.0,0.0
pool:DwarfPool,0.840000,0.120000,0.040000,0.0,0.0,0.0
pool:Ethermine,0.720000,0.150000,0.130000,0.0,0.0,0.0
...,...,...,...,...,...,...
pool:Suprnova,0.333333,0.333333,0.333333,0.0,0.0,0.0
pool:Uleypool,0.000000,0.000000,0.000000,0.0,1.0,0.0
pool:WaterholePool,0.000000,0.000000,0.000000,1.0,0.0,0.0
pool:Whalesburg Pool,1.000000,0.000000,0.000000,0.0,0.0,0.0


In [291]:
### Maybe we don't need this step (unless we want to go granular with locations) 
# this produces a table of granular locations (eg: us-east, us-west), with the percentages assigned to each
# and creates the indexes in the same format as in the df (pool:NameOfPool, extraData:location)

# load mapping from a region to distribution across grid, and normalize
region_to_grid_mix = pd.read_csv('input/region-to-grid-mix.csv').set_index('region')
region_to_grid_mix = normalize_row_to_one(region_to_grid_mix.fillna(0))

region_label_to_region_mix = pd.DataFrame({region:{region:1} for region in region_to_grid_mix.index})
region_label_to_region_mix.index = ['extraData:' + e for e in region_label_to_region_mix.index]
label_to_region_mix = pd.concat((miner_label_to_region_mix, region_label_to_region_mix)).fillna(0)

# manually add a row for 'unknown' label mapping it to 'unknown' region
# warning every time you run it, you are adding a new 'unknown' row - so first check if the index exists
if ('unknown' not in label_to_region_mix.index):
    label_to_region_mix = label_to_region_mix.append(pd.Series(0, index=label_to_region_mix.columns, name='unknown'))
    label_to_region_mix.loc['unknown', 'unknown'] = 1
label_to_region_mix
# these numbers mean that the pool (eg pool:2Miners) can be assigned 
# 95% of the times (0.95) to Europe, 4% of the times to USA, and 1% of the times to China
# this is based on previous research

,europe,us,asia,china,russia,unknown,singapore,taiwan,seoul,europe-west,europe-north,ukraine,us-east,us-west
pool:2Miners,0.95,0.04,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:Bw Pool,0.32,0.10,0.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:Coinotron,0.92,0.08,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:DwarfPool,0.84,0.12,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:Ethermine,0.72,0.15,0.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
extraData:ukraine,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
extraData:us,0.00,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
extraData:us-east,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
extraData:us-west,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Utility Functions

- get_all_with_location_label(_df, label)
- assign_location_for_label_name(_df, locations_df, label)
- assign_manual_location_to_blocks_with_label(df, 'extraData:russia', 'europe)

In [292]:
### ---------------------------------------  Utility Functions
def get_all_with_location_label(_df, label):
    subset = _df[_df['location'] == label]
    #print(f'location label: {label} was found {subset.count()[0]} times\n')
    #print(subset.info())
    return subset.copy() #always return a copy not the original df
    
#get_all_with_location_label(df, 'pool:F2Pool') 


In [293]:
# allows to assign a manual location to a label
# run like this
# assign_manual_location_to_blocks_with_label(df, 'extraData:russia', 'europe
'''
part = df[df['location'] == 'extraData:russia'].copy() # 1 - select a subset and turn it into a df
part.loc[:, 'location_assigned'] = 'europe' # 2- update it's values
df.update(pool_df)                          # 3- merge it back into df
'''
def assign_manual_location_to_blocks_with_label(_df, label_to_find, new_location):
    part = _df[_df['location'] == label_to_find].copy()   # 1 - select a subset and turn it into a df
    part.loc[:, 'location_assigned'] = new_location       # 2- update it's values
    df.update(part)                                       # 3- merge it back into df
    print(f"updated {label_to_find} >> changed to >> {new_location}")

# verify 
# df[df['location'] == 'extraData:russia']

### Method -- assign_location_for_label_name

In [294]:
# making a utility function to call all this in one line
def assign_location_for_label_name(_df, locations_df, label, trace=True):
    t=trace
    
    pool_name = label   #'pool:F2Pool'
    
    # if this labels doesn't exist in the label_to_region_mix it throw an error -skip it but print it to the console to know how to treat it by hand
    if (label not in locations_df.index):
        print(f"location {label} WAS NOT FOUND in label_to_region_mix.index ")
        return
    
    #access a row in label_to_region_mix + drop all useless labels (that have 0% allocation)
    pool_locations = locations_df.loc[[pool_name]].replace(0,np.nan).dropna(axis=1,how="all")
    if t: print(pool_locations)

    # access all rows that have that pool  (based on the content of the variable "pool_name") using boolean indexing
    # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
    #pool_df = df[df['location'] == pool_name]
    pool_df = get_all_with_location_label(df, pool_name)
    
    # find how many blocks have that location label
    # 2 ways of getting the number or row (or length) in a DF
    if t: print(f"len(pool_df.index) = {len(pool_df.index)}")
    pool_items_count = pool_df.shape[0]   # eg 922

    # convert the pool_locations percentages to number of rows that need to be filled with that location, based on the length of pool_items_count
    #             europe   us  asia       turn to                    europe    us       asia
    #pool:F2Pool    0.23  0.1  0.67                   pool:F2Pool    0.23*922  0.1*922  0.67*922
    pool_locations_n = pool_locations.multiply(pool_items_count).round().astype(int)
    pool_locations_n
    if t: print(f"pool_locations:--------------\n{pool_locations}")
    if t: print(f"pool_locations_n:--------------\n{pool_locations_n}")

    # verify that the total matches pool_items_count - if it doesn't substract the difference from the minimum
    if (pool_locations_n.values.sum() != pool_items_count):
        count_found = pool_locations_n.values.sum()
        delta = pool_items_count - count_found
        print(f"\nWARNING: problem with location label {pool_locations_n.index}")
        print(f"expected pool_items_count: {pool_items_count}")
        print(f"calculated count_found: {count_found}")
        print(f"delta: {delta}")
        
        #delete these from the smallest or the first one if there are multiple min values
        min_val = pool_locations_n.iloc[0].min()
        new_val = min_val + delta
        if (len(pool_locations_n.min().values) > 1):
            # this label has more than one sngle minimum value, just pick the first one to remove it from
            pool_locations_n.iloc[0,0] = new_val
        else:
            pool_locations_n = pool_locations_n.replace( min_val, new_val)   ## ths works only if there aren't multiple values that are the same
        print(f"pool_locations_n after correcting the delta:\n{pool_locations_n}")

    
    ## iterate over the pool_locations and assign the locations to the blocks
    ## assign 212 blocks to the europe label, 92 to the european and 618 to asia
    ##               europe  us   asia
    ## pool:F2Pool     212   92   618

    #change the current index based on block_number to a numeric index to be able to easily target rows for change
    # adds an index from 0,1,2 ...
    pool_df = pool_df.reset_index().rename(columns={"index":"block_number"})
    #print(pool_df.head())
    
    #print(f"\niterating over\n{pool_locations_n}")
    index_start = 0
    for col, row in pool_locations_n.iteritems():
        location = col
        amount = row[0]
    
        #select a range of items and change the values of the "location_assigned" column
        #index_start = 2
        index_end = index_start + amount
        if t: print(f"assigning {amount} to {location} - from index: {index_start} to index: {index_end}")
        pool_df.loc[index_start:index_end, 'location_assigned'] = location
        index_start = index_end+1
    
    ## Merge the new pool_df filled into the main df, so that all labels are transfered

    # to merge we need first to return the index of pool_df to be the block_number like in the main df
    # but do it only once otherwise jupyther will throw an error
    if('block_number' in pool_df.columns):
        #print("changing index and name")
        pool_df = pool_df.set_index("block_number").rename_axis(index=None)

    # now merge the pool_df data into the old
    df.update(pool_df)
    
    if t: print(f"\n assigned all location values to -- {label} --") 
    #print(f"\n----------- pool_df.shape(): {pool_df.shape}")
    #print(f"pool_df:\n {pool_df.head()}")
    #print(f"\n----------- df.shape():      {df.shape}")
    #print(f"df:     \n {df.head()}")

In [295]:
# testing it and solving problemms
# label_str = 'pool:Flexpool.io'
label_str   = 'pool:MiningPoolHub' # causes miscalculations #'pool:K1POOL.COM' 
assign_location_for_label_name(df, label_to_region_mix, label_str )
test = get_all_with_location_label(df, label_str)
test

                      europe        us      asia
pool:MiningPoolHub  0.333333  0.333333  0.333333
len(pool_df.index) = 7
pool_locations:--------------
                      europe        us      asia
pool:MiningPoolHub  0.333333  0.333333  0.333333
pool_locations_n:--------------
                    europe  us  asia
pool:MiningPoolHub       2   2     2

expected pool_items_count: 7
calculated count_found: 6
delta: 1
pool_locations_n after correcting the delta:
                    europe  us  asia
pool:MiningPoolHub       3   2     2
assigning 3 to europe - from index: 0 to index: 3
assigning 2 to us - from index: 4 to index: 6
assigning 2 to asia - from index: 7 to index: 9

 assigned all location values to -- pool:MiningPoolHub --


,extraData,location,location_assigned
15532133,vir1,pool:MiningPoolHub,europe
15533093,vir1,pool:MiningPoolHub,europe
15533235,vir1,pool:MiningPoolHub,europe
15533244,fra1,pool:MiningPoolHub,europe
15534647,fra1,pool:MiningPoolHub,us
15535184,fra1,pool:MiningPoolHub,us
15537310,vir1,pool:MiningPoolHub,us


## Parse all rows/ block_numbers and assign locations

In [296]:
label_to_region_mix.head()

,europe,us,asia,china,russia,unknown,singapore,taiwan,seoul,europe-west,europe-north,ukraine,us-east,us-west
pool:2Miners,0.95,0.04,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:Bw Pool,0.32,0.10,0.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:Coinotron,0.92,0.08,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:DwarfPool,0.84,0.12,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pool:Ethermine,0.72,0.15,0.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [297]:
## BB MOD we don't need the region mix in the equation because we only take the electricity consumption
# now we have 
#1) a   label_to_region_mix   that contains in the 'colums' the name of the region that needs to be assigned to a specific pool (in its index)
#2) a percetage of times it should be assigned
#3) df, our dataframe of block_number | location | information

# we need to extract from the df dataframe all pool locations 
# and allocate it in a certain percentage specified in  'label_to_region_mix' to that specific region
# I'm sure there is a pandas ultra compact way of doing it with matrix multiplications but can't make it work

parsed_locations = [] #will contain the list of names that we've already assigned so we can skip them if already in it
#dfg = df.groupby(['location'])
print(parsed_locations)

def parse_location_label(row):
    block_number = row.name
    location = row['location']
    #print(f"parse row: {row}")
    #print(f"parse block_number: {block_number} - {location} - \talready parsed: {location in parsed_locations}")
    
    if location not in parsed_locations:
        assign_location_for_label_name(df, label_to_region_mix, location, trace=False)
        parsed_locations.append(location)


#change the index to a normal progressive so you can get also the block_number if you need it

#df_full = df.head().apply(lambda row : parse_location_label(row), axis=1)  # for testing run only on the head()
df_full = df.apply(lambda row : parse_location_label(row), axis=1)
print(f"\nfinished parsing {len(parsed_locations)} locations:\n{parsed_locations}")
                         
df


[]
location extraData:asia-east WAS NOT FOUND in label_to_region_mix.index 

expected pool_items_count: 19
calculated count_found: 18
delta: 1
pool_locations_n after correcting the delta:
                 europe  us  china
pool:K1POOL.COM       7   6      6

expected pool_items_count: 7
calculated count_found: 6
delta: 1
pool_locations_n after correcting the delta:
                    europe  us  asia
pool:MiningPoolHub       3   2     2

finished parsing 24 locations:
['extraData:asia', 'unknown', 'pool:F2Pool', 'pool:Flexpool.io', 'extraData:singapore', 'extraData:asia-east', 'extraData:europe', 'extraData:russia', 'pool:Hiveon', 'extraData:seoul', 'extraData:us-west', 'extraData:us-east', 'extraData:us', 'pool:K1POOL.COM', 'extraData:europe-north', 'extraData:europe-west', 'pool:ETH.SoloPool.org', 'pool:Nanopool', 'pool:Ethermine', 'pool:MiningPoolHub', 'extraData:china', 'pool:Minerall Pool', 'pool:ZET Technologies', 'extraData:taiwan']


,extraData,location,location_assigned
15531512,EthereumPPLNS/2miners_ASIA4,extraData:asia,asia
15531513,lhpool.com-h15,unknown,unknown
15531514,七彩神仙鱼F !,pool:F2Pool,europe
15531515,七彩神仙鱼F#!,pool:F2Pool,europe
15531516,Flexpool/S1/HK - Boston,pool:Flexpool.io,europe
...,...,...,...
15537389,asia-east2-11,extraData:asia-east,NaN
15537390,七彩神仙鱼G!!,pool:F2Pool,asia
15537391,"七彩神仙鱼K.""",pool:F2Pool,asia
15537392,asia-east2-1v92,extraData:asia-east,NaN


In [298]:
df

,extraData,location,location_assigned
15531512,EthereumPPLNS/2miners_ASIA4,extraData:asia,asia
15531513,lhpool.com-h15,unknown,unknown
15531514,七彩神仙鱼F !,pool:F2Pool,europe
15531515,七彩神仙鱼F#!,pool:F2Pool,europe
15531516,Flexpool/S1/HK - Boston,pool:Flexpool.io,europe
...,...,...,...
15537389,asia-east2-11,extraData:asia-east,NaN
15537390,七彩神仙鱼G!!,pool:F2Pool,asia
15537391,"七彩神仙鱼K.""",pool:F2Pool,asia
15537392,asia-east2-1v92,extraData:asia-east,NaN


## Allocate some locations by 'hand' (russia, asia-east, unknown)
- Assign as
ia-east to China
- Assign Russia to Europe
- for 'unknown' do a diistribution of the other location percentages

In [299]:
## Assign Russia to Europe and asia-east to china
assign_manual_location_to_blocks_with_label(df, 'extraData:russia', 'europe')
assign_manual_location_to_blocks_with_label(df, 'extraData:europe-west', 'europe')
assign_manual_location_to_blocks_with_label(df, 'extraData:europe-north', 'europe')

assign_manual_location_to_blocks_with_label(df, 'extraData:asia-east', 'china')
assign_manual_location_to_blocks_with_label(df, 'extraData:asia', 'china')

updated extraData:russia >> changed to >> europe
updated extraData:europe-west >> changed to >> europe
updated extraData:europe-north >> changed to >> europe
updated extraData:asia-east >> changed to >> china
updated extraData:asia >> changed to >> china


## Parse the 'unknown' locations and assign to a distribution of the rest of locations

In [300]:
tot_unknown = df['location_assigned'].value_counts()['unknown']
tot_unknown

1410

In [301]:
# find which and how many are known
#df['location_assigned'].value_counts().sum() # - works - 5882 - add them up
locations_known = df['location_assigned'].value_counts().drop('unknown')
tot_known = locations_known.sum()
print(f'tot_known: {tot_known}')
locations_known#.values

tot_known: 4472


europe       1642
china        1031
asia          616
us            425
us-east       311
singapore     194
seoul         127
us-west       125
taiwan          1
Name: location_assigned, dtype: int64

In [302]:
## build a dataframe to host the % of that amount
columns = {
    'amount': locations_known.values,
    'percent': np.nan,
    'amount_to_assign': np.nan
}
locations_known_df = pd.DataFrame(columns, index=locations_known.index)
#df = pd.DataFrame(columns, index=blocks_for_dataFrame['block_number'])
#locations_known_df

## assign the percentage =  amount/tot_known x
locations_known_df.loc[:,'percent'] = locations_known_df.loc[:,'amount'] / tot_known
#locations_known_df['percent'].values.sum() # verfy that sums to 1

## add the colum amount_to_assign
locations_known_df.loc[:,'amount_to_assign'] = round(locations_known_df.loc[:,'percent'] * tot_unknown).astype(int)

locations_known_df

,amount,percent,amount_to_assign
europe,1642,0.367174,518
china,1031,0.230546,325
asia,616,0.137746,194
us,425,0.095036,134
us-east,311,0.069544,98
singapore,194,0.043381,61
seoul,127,0.028399,40
us-west,125,0.027952,39
taiwan,1,0.000224,0


In [303]:
## verify that we didn't forget to assign one and add it to the biggest
tot_assigned = locations_known_df.loc[:,'amount_to_assign'].values.sum()
print(f"tot_assigned: {tot_assigned}")
if ( tot_assigned != tot_unknown):
    delta = tot_unknown - tot_assigned
    print(f"tot_unknown: {tot_unknown} - tot_assigned: {tot_assigned} = delta: {delta} unknown")
    # assign the missing one to the most common item
    max_value = locations_known_df.loc[:,'amount_to_assign'].max()
    if type(max_value) == int:      # only one max value exists
        # we are adding it to the first value   values[0]  because .value_counts() already returns it sorted descending
        locations_known_df.loc[:,'amount_to_assign'].values[0] += delta    
    else:                           # it might be an array
        max_value[0] += delta 
    
    tot_assigned = locations_known_df.loc[:,'amount_to_assign'].values.sum() #verify again
    print(f'tot_assigned after correction: {tot_assigned}')
    
locations_known_df

tot_assigned: 1409
tot_unknown: 1410 - tot_assigned: 1409 = delta: 1 unknown
tot_assigned after correction: 1410


,amount,percent,amount_to_assign
europe,1642,0.367174,519
china,1031,0.230546,325
asia,616,0.137746,194
us,425,0.095036,134
us-east,311,0.069544,98
singapore,194,0.043381,61
seoul,127,0.028399,40
us-west,125,0.027952,39
taiwan,1,0.000224,0


In [304]:
 ### Iterate over the unknowns and assign

unknown_set = df[df['location_assigned'] == 'unknown'].copy()

#change the current index based on block_number to a numeric index to be able to easily target rows for change
# adds an index from 0,1,2 ...
unknown_set = unknown_set.reset_index().rename(columns={"index":"block_number"})
print(unknown_set.head())

# iterate over the locations_known_df and assign the value to the unknown set
index_start = 0
for location in locations_known_df.index:
    amount = locations_known_df.loc[location,'amount_to_assign']
    index_end = index_start + amount
    print(f"assigning {amount} to {location} - from index: {index_start} to index: {index_end}")
    unknown_set.loc[index_start:index_end, 'location_assigned'] = location
    index_start = index_end+1

#unknown_set.head()

## check if there are still unknown
print(f"\n----- no more unknowns in the set?: {('unknown' not in unknown_set['location_assigned'].value_counts().index)}")  ##expected no unknown in the list


   block_number             extraData location location_assigned
0      15531513        lhpool.com-h15  unknown           unknown
1      15531526  poolin.com %L\nn±R8þ  unknown           unknown
2      15531531            4hash GmbH  unknown           unknown
3      15531533  poolin.com Ô´mk\nÑ  unknown           unknown
4      15531535   poolin.com!8õßÀ  unknown           unknown
assigning 519 to europe - from index: 0 to index: 519
assigning 325 to china - from index: 520 to index: 845
assigning 194 to asia - from index: 846 to index: 1040
assigning 134 to us - from index: 1041 to index: 1175
assigning 98 to us-east - from index: 1176 to index: 1274
assigning 61 to singapore - from index: 1275 to index: 1336
assigning 40 to seoul - from index: 1337 to index: 1377
assigning 39 to us-west - from index: 1378 to index: 1417
assigning 0 to taiwan - from index: 1418 to index: 1418

----- no more unknowns in the set?: True


In [305]:
#merge the new unknown_set filled into the main df so that all labels are transferred

# to merge we need first to return the index of pool_df to be the block_number like in the main df
# but do it only once otherwise jupyther will throw an error
if('block_number' in unknown_set.columns):
    #print("changing index and name")
    unknown_set = unknown_set.set_index("block_number").rename_axis(index=None)

# now merge the pool_df data into the old
df.update(unknown_set)
print(f"\n assigned all location values to -- 'unknown' --") 


 assigned all location values to -- 'unknown' --


In [306]:
## verify - this should be empty
df[df['location_assigned'] == 'unknown']

,extraData,location,location_assigned


## EXPORT CSV + JSON


### WIP (UNFINNISHED) sanitize certain extra_daata fields or drop them for the CSV 
because when exporting to csv they create new lines<br>
eg block_number:15531544<br>has extra_data: "\u4e03\u5f69\u795e\u4ed9\u9c7cH\r!",<br>
the last "\r" creates a new line in the csv preview<br>
but this doesn't happen in the JSON<br>

managed to replace \r but not \x or other strings

In [349]:
### debugging some problems in some records eg 15531544 and the one after
start = 15531567
#df.loc[start:start+1]
weird_data = df.loc[start, 'extraData']
weird_data

'poolin.com Òô\x0bö_®\x01\xa0'

In [354]:
weird_data.encode()
weird_bytes = bytearray(weird_data.encode())
weird_bytes

bytearray(b'poolin.com \xc3\x92\xc3\xb4\x0b\xc3\xb6_\xc2\xae\x01\xc2\xa0')

In [355]:
from block_index import decode_extra_data
decode_extra_data(weird_bytes)

'poolin.com Òô\x0bö_®\x01\xa0'

In [368]:
# SANITIZE before exporting CSV
# cleans the \r fromm strings like the extradata of block 15531543
# turning  '七彩神仙鱼H\r!'  into '七彩神仙鱼H_r!'
# replace the string in all of the dataframe
#df.replace('\r',"_r", inplace=True, regex=True)   #working
#df.replace(r'\\x',"_x", inplace=True, regex=True) #not working
#df.replace(r'\\',"_", inplace=True, regex=True) 
#df = df.replace(to_replace= r'\\', value= '', regex=True) # not working

for ed in df.loc[:, 'extraData'].values:
    ed.replace(r"\\","")

# verify it you still see it here -  it should show '七彩神仙鱼H_r!'
df.loc[start, 'extraData']

'poolin.com Òô\x0bö_®\x01\xa0'

### EXPORT THE FILES

europe       2162
china        1357
asia          811
us            560
us-east       410
singapore     256
seoul         168
us-west       157
taiwan          1
Name: location_assigned, dtype: int64

In [373]:
## EXPORT THE FILES
filename = 'Day1_locations'
df.to_csv(f'output/{filename}.csv')
df.to_json(f'output/{filename}.json')
df.to_excel(f'output/{filename}.xls')


#Build a summary of location > amounts for 
df['location_assigned'].value_counts().to_csv(f'output/{filename}_groupedByLocation.csv')

/Users/b/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """


# CODE TO ARCHIVE

In [371]:
#Methods of pandas Series
dir(df['location_assigned'].value_counts())

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r

In [314]:
df.loc[df['location_assigned'] != 'unknown', 'location_assigned']

15531512     china
15531513    europe
15531514    europe
15531515    europe
15531516    europe
             ...  
15537389     china
15537390      asia
15537391      asia
15537392     china
15537393      asia
Name: location_assigned, Length: 5882, dtype: object

In [315]:
unknowns_df = get_all_with_location_label(df, 'unknown')
df.describe()


,extraData,location,location_assigned
count,5882,5882,5882
unique,1761,24,9
top,nanopool.org,unknown,europe
freq,157,1410,2162


In [316]:
##### TESTING TO SOLVE

label_to_find = 'extraData:asia-east'
subset = df[df['location'] == label_to_find]
subset
lf = label_to_region_mix 
if (label_to_find not in lf.index):
    print(f"location {label_to_find} WAS NOT FOUND in label_to_region_mix ")
#lf[lf['location'] == label_to_find]

location extraData:asia-east WAS NOT FOUND in label_to_region_mix 


In [317]:
label_to_region_mix.index

Index(['pool:2Miners', 'pool:Bw Pool', 'pool:Coinotron', 'pool:DwarfPool',
       'pool:Ethermine', 'pool:F2Pool', 'pool:firepool', 'pool:Hiveon',
       'pool:Huobi Mining Pool', 'pool:Nanopool', 'pool:PandaPool',
       'pool:Spark Pool', 'pool:UUPool', 'pool:xnpool.cn', 'pool:zhizhu.top',
       'pool:AlphaPool', 'pool:EthereumPool', 'pool:ExtremeHash',
       'pool:HashON Pool', 'pool:Huixingpool.com', 'pool:Weipool',
       'pool:AntPool', 'pool:Babel Pool', 'pool:BaikalMine 1', 'pool:Baypool',
       'pool:BeePool', 'pool:BitClubPool', 'pool:BTC.com Pool',
       'pool:CoinMine.pl', 'pool:CoolPool.Top', 'pool:Cruxpool',
       'pool:DigiPools', 'pool:Easy2Mine', 'pool:Eth.pp.ua',
       'pool:ETH.SoloPool.org', 'pool:EthashPool', 'pool:EtherDig',
       'pool:ethfans.org', 'pool:Ethpool', 'pool:EzilPool', 'pool:FKPool',
       'pool:Flexpool.io', 'pool:Genesis Mining', 'pool:ICanMining.ru',
       'pool:K1POOL.COM', 'pool:KuveraPool', 'pool:MATPool', 'pool:MaxHash',
       'pool:

### Testing iteritems()

In [318]:
### TEST
students = [('Ankit', 22, 'A'),
           ('Swapnil', 22, 'B'),
           ('Priya', 22, 'B'),
           ('Shivangi', 22, 'B'),
            ]
 
# Create a DataFrame object
stu_df = pd.DataFrame(students, columns =['Name', 'Age', 'Section'],
                      index =['1', '2', '3', '4'])
 
stu_df

,Name,Age,Section
1,Ankit,22,A
2,Swapnil,22,B
3,Priya,22,B
4,Shivangi,22,B


In [319]:
# gives a tuple of column name and series
# for each column in the dataframe
for (columnName, columnData) in stu_df.iteritems():
    print('Column Name : ', columnName)
    print('Column Contents : ', columnData.values)

Column Name :  Name
Column Contents :  ['Ankit' 'Swapnil' 'Priya' 'Shivangi']
Column Name :  Age
Column Contents :  [22 22 22 22]
Column Name :  Section
Column Contents :  ['A' 'B' 'B' 'B']


In [320]:
for column in stu_df:
     
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = stu_df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

Column Name :  Name
Column Contents :  ['Ankit' 'Swapnil' 'Priya' 'Shivangi']
Column Name :  Age
Column Contents :  [22 22 22 22]
Column Name :  Section
Column Contents :  ['A' 'B' 'B' 'B']


In [321]:
# Iterate over the index range from
# 0 to max number of columns in dataframe
for index in range(stu_df.shape[1]):
    
    print('Column Number : ', index)
     
    # Select column by index position using iloc[]
    columnSeriesObj = stu_df.iloc[:, index]
    print('Column Contents : ', columnSeriesObj.values)

Column Number :  0
Column Contents :  ['Ankit' 'Swapnil' 'Priya' 'Shivangi']
Column Number :  1
Column Contents :  [22 22 22 22]
Column Number :  2
Column Contents :  ['A' 'B' 'B' 'B']


### Tests - process a Pool_name (works - finished and added to assign_location_for_label_name()
maybe turn into a utility function to process all location labels, both pool: and extraData:

In [322]:
# TEST
#access a row in label_to_region_mix + drop all useless labels (that have 0% allocation)
pool_name = 'pool:F2Pool'
pool_locations = label_to_region_mix.loc[[pool_name]].replace(0,np.nan).dropna(axis=1,how="all")
print(pool_locations)
print(type(pool_locations))

# access all rows that have that pool  (based on the content of the variable "pool_name") using boolean indexing
# https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
#pool_df = df[df['location'] == pool_name]
pool_df = get_all_with_location_label(df, pool_name)
pool_df

             europe   us  asia
pool:F2Pool    0.23  0.1  0.67
<class 'pandas.core.frame.DataFrame'>


,extraData,location,location_assigned
15531514,七彩神仙鱼F !,pool:F2Pool,europe
15531515,七彩神仙鱼F#!,pool:F2Pool,europe
15531520,七彩神仙鱼K+!,pool:F2Pool,europe
15531525,七彩神仙鱼J!,pool:F2Pool,europe
15531543,七彩神仙鱼J'!,pool:F2Pool,europe
...,...,...,...
15537351,七彩神仙鱼H.!,pool:F2Pool,asia
15537382,七彩神仙鱼I$!,pool:F2Pool,asia
15537390,七彩神仙鱼G!!,pool:F2Pool,asia
15537391,"七彩神仙鱼K.""",pool:F2Pool,asia


In [323]:
# find how many blocks have that location label
# 2 ways of getting the number or row (or length) in a DF
# print(f"pool_df.shape[0] = {pool_df.shape[0]}")
print(f"len(pool_df.index) = {len(pool_df.index)}")
pool_items_count = pool_df.shape[0]   # eg 922

# convert the pool_locations percentages to number of rows that need to be filled with that location, based on the length of pool_items_count
#             europe   us  asia       turn to                    europe    us       asia
#pool:F2Pool    0.23  0.1  0.67                   pool:F2Pool    0.23*922  0.1*922  0.67*922
pool_locations_n = pool_locations.multiply(pool_items_count).round().astype(int)
pool_locations_n
print(f"\npool_locations:--------------\n{pool_locations}")
print(f"\npool_locations_n:--------------\n{pool_locations_n}")

# verify that the total matches pool_items_count - if it doesn't substract from the minimum
if (pool_locations_n.values.sum() != pool_items_count):
    count_found = pool_locations_n.values.sum()
    delta = abs(pool_items_count - count_found)
    print(f"\nWARNING: problem with location label {pool_locations_n.index}")
    print(f"expected pool_items_count: {pool_items_count}")
    print(f"calculated count_found: {count_found}")
    #delete these from the smallest
    min_val = pool_locations_n.iloc[0].min()
    new_val = min_val - delta
    pool_locations_n = pool_locations_n.replace( min_val, new_val)
    print(pool_locations_n)

    

len(pool_df.index) = 922

pool_locations:--------------
             europe   us  asia
pool:F2Pool    0.23  0.1  0.67

pool_locations_n:--------------
             europe  us  asia
pool:F2Pool     212  92   618


In [324]:
## iterate over the pool_locations and assign the locations to the blocks
## assign 212 blocks to the europe label, 92 to the european and 618 to asia
##               europe  us   asia
## pool:F2Pool     212   92   618

#change the current index based on block_number to a numeric index to be able to easily target rows for change
# adds an index from 0,1,2 ...
pool_df = pool_df.reset_index().rename(columns={"index":"block_number"})
#print(pool_df.head())
pool_df


,block_number,extraData,location,location_assigned
0,15531514,七彩神仙鱼F !,pool:F2Pool,europe
1,15531515,七彩神仙鱼F#!,pool:F2Pool,europe
2,15531520,七彩神仙鱼K+!,pool:F2Pool,europe
3,15531525,七彩神仙鱼J!,pool:F2Pool,europe
4,15531543,七彩神仙鱼J'!,pool:F2Pool,europe
...,...,...,...,...
917,15537351,七彩神仙鱼H.!,pool:F2Pool,asia
918,15537382,七彩神仙鱼I$!,pool:F2Pool,asia
919,15537390,七彩神仙鱼G!!,pool:F2Pool,asia
920,15537391,"七彩神仙鱼K.""",pool:F2Pool,asia


In [325]:
print(f"\niterating over\n{pool_locations_n}")
index_start = 0
for col, row in pool_locations_n.iteritems():
    location = col
    amount = row[0]
    
    #select a range of items and change the values of the "location_assigned" column
    #index_start = 2
    index_end = index_start + amount
    print(f"assigning {amount} to {location} - from index: {index_start} to index: {index_end}")
    pool_df.loc[index_start:index_end, 'location_assigned'] = location
    index_start = index_end+1

pool_df


iterating over
             europe  us  asia
pool:F2Pool     212  92   618
assigning 212 to europe - from index: 0 to index: 212
assigning 92 to us - from index: 213 to index: 305
assigning 618 to asia - from index: 306 to index: 924


,block_number,extraData,location,location_assigned
0,15531514,七彩神仙鱼F !,pool:F2Pool,europe
1,15531515,七彩神仙鱼F#!,pool:F2Pool,europe
2,15531520,七彩神仙鱼K+!,pool:F2Pool,europe
3,15531525,七彩神仙鱼J!,pool:F2Pool,europe
4,15531543,七彩神仙鱼J'!,pool:F2Pool,europe
...,...,...,...,...
917,15537351,七彩神仙鱼H.!,pool:F2Pool,asia
918,15537382,七彩神仙鱼I$!,pool:F2Pool,asia
919,15537390,七彩神仙鱼G!!,pool:F2Pool,asia
920,15537391,"七彩神仙鱼K.""",pool:F2Pool,asia


In [326]:
## Merge the new pool_df filled into the main df, so that all labels are transfered

# to merge we need first to return the index of pool_df to be the block_number like in the main df
# but do it only once otherwise jupyther will throw an error
if('block_number' in pool_df.columns):
    print("changing index and name")
    pool_df = pool_df.set_index("block_number").rename_axis(index=None)

# now merge the pool_df data into the old
df.update(pool_df)
print(f"\n----------- pool_df.shape(): {pool_df.shape}")
print(f"pool_df:\n {pool_df.head()}")
print(f"\n----------- df.shape():      {df.shape}")
print(f"df:     \n {df.head()}")

changing index and name

----------- pool_df.shape(): (922, 3)
pool_df:
          extraData     location location_assigned
15531514  七彩神仙鱼F !  pool:F2Pool            europe
15531515  七彩神仙鱼F#!  pool:F2Pool            europe
15531520  七彩神仙鱼K+!  pool:F2Pool            europe
15531525  七彩神仙鱼J!  pool:F2Pool            europe
15531543  七彩神仙鱼J'!  pool:F2Pool            europe

----------- df.shape():      (5882, 3)
df:     
                             extraData          location location_assigned
15531512  EthereumPPLNS/2miners_ASIA4    extraData:asia             china
15531513               lhpool.com-h15           unknown            europe
15531514                     七彩神仙鱼F !       pool:F2Pool            europe
15531515                     七彩神仙鱼F#!       pool:F2Pool            europe
15531516      Flexpool/S1/HK - Boston  pool:Flexpool.io            europe


In [327]:
'''
# load block labels
block_labels = read_csv_date('output/block-labels.csv').fillna(0)
date_to_regions_with_unknown = dot(block_labels, label_to_region_mix)
date_to_regions_with_unknown
'''

"\n# load block labels\nblock_labels = read_csv_date('output/block-labels.csv').fillna(0)\ndate_to_regions_with_unknown = dot(block_labels, label_to_region_mix)\ndate_to_regions_with_unknown\n"